In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns

In [3]:
pcds_path = Path("/DATA/Luis/thesis/scanline_classification/data/data_for_training/scanline_subsampling/merged/training_data_merged_all.csv")

label_names={0: "unclassified",
             1: "man-made objects",
             2: "ground",
             3: "tree trunk/branches",
             4: "leaves",
             5: "low vegetation"}

# colors_dict = {"man-made objects": sns.color_palette("husl", 4)[0],
#                "ground": sns.color_palette("husl", 4)[3],
#                "tree trunk/branches": sns.color_palette("husl", 4)[2],
#                "leaves": sns.color_palette("husl", 4)[1]}

pcd_df = pd.read_csv(pcds_path, sep=",", header=0)
pcd_df['label_names'] = pcd_df['label'].map(label_names)

In [5]:
drop_cols = ["x_median_nn", "y_median_nn", "x_perc2nd_nn", "y_perc2nd_nn", "x_perc98th_nn", "y_perc98th_nn", "segment_id"]
pcd_df_frac = pcd_df.drop(columns=drop_cols)

In [7]:
# Remove the unclassified and low vegetation points
pcd_df_frac = pcd_df_frac.groupby('label_names').sample(n=100000, replace=False, random_state=42)

In [8]:
pcd_df_frac['label_names'].value_counts()

label_names
ground                 100000
leaves                 100000
low vegetation         100000
man-made objects       100000
tree trunk/branches    100000
unclassified           100000
Name: count, dtype: int64

In [10]:
pcd_df_frac.columns

Index(['z_median_nn', 'z_perc2nd_nn', 'z_perc98th_nn', 'intensity_mean',
       'intensity_var', 'intensity_std', 'intensity_median',
       'intensity_perc2nd', 'intensity_perc98th', 'intensity_skewness',
       ...
       'ny_skewness', 'nz_mean', 'nz_var', 'nz_std', 'nz_median', 'nz_perc2nd',
       'nz_perc98th', 'nz_skewness', 'label', 'label_names'],
      dtype='object', length=103)

In [2]:
path = "/DATA/Luis/thesis/scanline_classification/data/data_for_training_z_normalization/validation/merged/validation_data_merged_frac.csv"

df = pd.read_csv(path, sep=",", header=0)

In [29]:
df.columns[df.columns.str.contains('intensity|red|green|curvature|roughness|vert_angle|nx|ny|nz', case=True) & 
           ~df.columns.str.contains('nx_xyz|ny_xyz', case=True)]

Index(['intensity_mean', 'intensity_var', 'intensity_std', 'intensity_median',
       'intensity_perc2nd', 'intensity_perc98th', 'intensity_perc25th',
       'intensity_perc75th', 'intensity_skewness', 'red_mean', 'red_var',
       'red_std', 'red_median', 'red_perc2nd', 'red_perc98th', 'red_perc25th',
       'red_perc75th', 'red_skewness', 'green_mean', 'green_var', 'green_std',
       'green_median', 'green_perc2nd', 'green_perc98th', 'green_perc25th',
       'green_perc75th', 'green_skewness', 'vert_angle_mean', 'vert_angle_var',
       'vert_angle_std', 'vert_angle_median', 'vert_angle_perc2nd',
       'vert_angle_perc98th', 'vert_angle_perc25th', 'vert_angle_perc75th',
       'vert_angle_skewness', 'curvature_mean', 'curvature_var',
       'curvature_std', 'curvature_median', 'curvature_perc2nd',
       'curvature_perc98th', 'curvature_perc25th', 'curvature_perc75th',
       'curvature_skewness', 'roughness_mean', 'roughness_var',
       'roughness_std', 'roughness_median', 'rough

In [8]:
df.columns[df.columns.str.find('intensity')]

Index(['label_names', 'label_names', 'label_names', 'label_names',
       'label_names', 'label_names', 'label_names', 'label_names',
       'label_names', 'label_names',
       ...
       'label_names', 'label_names', 'label_names', 'label_names',
       'label_names', 'label_names', 'label_names', 'label_names',
       'label_names', 'label_names'],
      dtype='object', length=149)

# 02.03.2024

## PCD classification on 3D data

- Import pcd txt-file and create an o3d object of it
- Subsample the pcd
    - Then convert back to a numpy array 
    - The subsampled pcd array should be linked with the original pcd array. Means only keep the points (rows) from the original pcd array that are in the subsampled pcd array
- Based on the original and subsampled pcd array calculate a kd-tree
    - For the beginning start with a neighborhood radius of 0.5m
- For each point and its neighborhood consider the following attributes:
    - Normalized Z (col 2), Intensity (3), Red(4), Green(5), Blue(6), Rho(7), theta(8) ...
    - And calculate the for each neighborhood the and each attribut the following statistics:
    - Mean, Median, Standard deviation, Variance, 2nd percentile, 25th percentile, 75th percentile and 98 percentile
    - Then also calculate the normals, the curvature and the roughness using a covariance matrix from the neighborhood and again calculate for each attribute (normals, curvature and roughness) the statistics left above
- Write all calculated aatributes/statistics to a new array
- Save the array along with the x,y,z values

In [2]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
pcd_path = "/DATA/Luis/thesis/scanline_classification/data/03_labeled/SiteA_RHV_01_Labeled.txt"
pcd = np.loadtxt(pcd_path, delimiter=" ")

NameError: name 'np' is not defined

In [3]:
pcd[0:10, -1]

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [13]:
# Duplizieren Sie pcd[:, -1] 9 mal
duplicated_pcd = np.repeat(pcd[:, -1], 9)

# Ändern Sie die Form von duplicated_pcd zu (pcd.shape[0], 3, 3)
reshaped_pcd = duplicated_pcd.reshape(pcd.shape[0], 3, 3)

In [18]:
reshaped_pcd[:,0,0].shape

(6989110,)

In [4]:
# Initialize the point cloud open3d object
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(pcd[:, :3])
pcd_o3d.colors = o3d.utility.Vector3dVector(pcd[:, (4,5,6)] / 255)
pcd_o3d.normals = o3d.utility.Vector3dVector(pcd[:, (3,7,8)])

In [14]:
subsampled_pcd = pcd_o3d.voxel_down_sample(voxel_size=0.1)

In [21]:
subsampled_pcd_array_xyz = np.asarray(subsampled_pcd.points)
subsampled_pcd_array_rgb = np.asarray(subsampled_pcd.colors) * 255
subsampled_pcd_array_refrhotht = np.asarray(subsampled_pcd.normals)

In [24]:
pcd[:, (4,5,6)] / 255

array([[0.74509804, 0.6       , 0.42352941],
       [0.74117647, 0.59607843, 0.41960784],
       [0.74117647, 0.59607843, 0.41960784],
       ...,
       [0.43921569, 0.4       , 0.40392157],
       [0.42745098, 0.38431373, 0.4       ],
       [0.41176471, 0.36862745, 0.38431373]])

In [22]:
subsampled_pcd_array_rgb

array([[ 85.77777778,  89.88888889,  79.77777778],
       [ 86.6       ,  88.51428571,  92.8       ],
       [ 69.5862069 ,  68.52873563,  70.86206897],
       ...,
       [163.42857143, 145.71428571, 127.42857143],
       [ 83.66666667,  73.66666667,  87.33333333],
       [ 76.8       ,  68.2       ,  81.4       ]])

In [1]:
from pathlib import Path

In [23]:
path = Path("/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/validation/SiteD_RHV_02_Labeled_Subsampled.txt")
segmentation_pcds_dir = Path("/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/scanline_segmentation")
file_to_search = "_".join(path.stem.split("_")[0:3])
segmentation_pcd = segmentation_pcds_dir.glob(f"{file_to_search}*").__next__()

print(segmentation_pcd)

/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/scanline_segmentation/SiteD_RHV_02_Labeled_Segmentation.txt


In [1]:
import pandas as pd

In [2]:
path = "/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/classification_training_sample10000/classification_report/SiteD_RHV_06_Labeled_Subsampled_nzcol_skewnessstat_cls_report_fr.csv"

In [3]:
df = pd.read_csv(path, sep=",", header=0)

In [8]:
df = df.rename(columns={df.columns[0]: "attribute"})

In [9]:
df

,attribute,precision,recall,f1-score,support,statistics,drop_col,baseline_model,filename
0,unclassified,0.061422,0.342944,0.104184,2.934300e+04,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
1,man-made objects,0.663999,0.306920,0.419797,1.151841e+06,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
2,ground,0.969571,0.962108,0.965825,4.155353e+06,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
3,tree trunk/branches,0.265002,0.440294,0.330865,6.853850e+05,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
4,leaves,0.714885,0.635577,0.672902,9.478020e+05,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
5,low vegetation,0.082164,0.528015,0.142200,3.109000e+04,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
6,accuracy,0.754494,0.754494,0.754494,7.544938e-01,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
7,macro avg,0.459507,0.535976,0.439296,7.000814e+06,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled
8,weighted avg,0.808090,0.754494,0.766898,7.000814e+06,skewness,nz,0,SiteD_RHV_06_Labeled_Subsampled


In [29]:
stats = ["mean","var","std","median","perc2nd","perc98th","perc25th","perc75th","skewness"]
col_to_consider = "z"
#cols_to_consider = "|".join(["_".join([col_to_consider, stat]) for stat in stats])
cols_to_consider = ["_".join([col_to_consider, stat]) for stat in stats]

In [30]:
cols_to_consider

['z_mean',
 'z_var',
 'z_std',
 'z_median',
 'z_perc2nd',
 'z_perc98th',
 'z_perc25th',
 'z_perc75th',
 'z_skewness']

In [19]:
import numpy as np

In [26]:
testt = np.array(['z_median_nn', 'z_perc2nd_nn', 'z_perc98th_nn', 'z_mean', 'z_var',
       'z_std', 'z_median', 'z_perc2nd', 'z_perc98th', 'z_perc25th',
       'z_perc75th', 'z_skewness', 'nx_xyz_mean', 'nx_xyz_var', 'nx_xyz_std',
       'nx_xyz_median', 'nx_xyz_perc2nd', 'nx_xyz_perc98th', 'nx_xyz_perc25th',
       'nx_xyz_perc75th', 'nx_xyz_skewness', 'ny_xyz_mean', 'ny_xyz_var',
       'ny_xyz_std', 'ny_xyz_median', 'ny_xyz_perc2nd', 'ny_xyz_perc98th',
       'ny_xyz_perc25th', 'ny_xyz_perc75th', 'ny_xyz_skewness', 'nz_xyz_mean',
       'nz_xyz_var', 'nz_xyz_std', 'nz_xyz_median', 'nz_xyz_perc2nd',
       'nz_xyz_perc98th', 'nz_xyz_perc25th', 'nz_xyz_perc75th',
       'nz_xyz_skewness', 'nz_mean', 'nz_var', 'nz_std', 'nz_median',
       'nz_perc2nd', 'nz_perc98th', 'nz_perc25th', 'nz_perc75th',
       'nz_skewness'])

In [28]:
list(set(cols_to_consider))

['v',
 'p',
 'w',
 'a',
 '|',
 '9',
 '8',
 'i',
 'm',
 '2',
 '5',
 'd',
 'z',
 '7',
 'h',
 '_',
 'r',
 'n',
 'e',
 't',
 'k',
 's',
 'c']

In [2]:
cols_to_consider = ["z","red","green","slope","curvature","roughness","nz_xyz","nx","nz"]
stats_to_consider = ["perc2nd","perc25th","perc75th","skewness"]

In [6]:
attributes_to_consider = [f"{i}_{j}" for i in cols_to_consider for j in stats_to_consider]

In [7]:
attributes_to_consider

['z_perc2nd',
 'z_perc25th',
 'z_perc75th',
 'z_skewness',
 'red_perc2nd',
 'red_perc25th',
 'red_perc75th',
 'red_skewness',
 'green_perc2nd',
 'green_perc25th',
 'green_perc75th',
 'green_skewness',
 'slope_perc2nd',
 'slope_perc25th',
 'slope_perc75th',
 'slope_skewness',
 'curvature_perc2nd',
 'curvature_perc25th',
 'curvature_perc75th',
 'curvature_skewness',
 'roughness_perc2nd',
 'roughness_perc25th',
 'roughness_perc75th',
 'roughness_skewness',
 'nz_xyz_perc2nd',
 'nz_xyz_perc25th',
 'nz_xyz_perc75th',
 'nz_xyz_skewness',
 'nx_perc2nd',
 'nx_perc25th',
 'nx_perc75th',
 'nx_skewness',
 'nz_perc2nd',
 'nz_perc25th',
 'nz_perc75th',
 'nz_skewness']

In [25]:
import pandas as pd
import numpy as np
import pickle

In [13]:
df1 = pd.DataFrame({"a": [1,1,2,2], "b": [0,0,0,0]})
df2 = pd.DataFrame({"a": [1,2], "d": [7,9]})    
df

,a,b
0,1,0
1,1,0
2,2,0
3,2,0


In [14]:
# join on the 'a' column
df1.merge(df2, on='a', how='inner')

,a,b,d
0,1,0,7
1,1,0,7
2,2,0,9
3,2,0,9


In [26]:
columns_paths = "/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/scanline_subsampling/attribute_statistics/SiteA_RHV_01_Labeled.pkl"

with open(columns_paths, "rb") as f:
    column_names = pickle.load(f)

In [29]:
subsampled_pcd = np.loadtxt("/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/validation/SiteD_RHV_01_Labeled_Subsampled.txt", delimiter=" ")
pcd = np.loadtxt("/DATA/Luis/thesis/scanline_classification/data/data_for_training_final_SM2CT20NM3_v02/scanline_segmentation/SiteD_RHV_01_Labeled_Segmentation.txt", delimiter=" ")

In [40]:
cols = ["id", "y_pred"]
subsampled_pcd_df = pd.DataFrame(subsampled_pcd[:, [-2, -1]], columns=cols)
subsampled_pcd_df

,id,y_pred
0,1.0,2.0
1,2.0,2.0
2,3.0,2.0
3,4.0,2.0
4,5.0,2.0
...,...,...
484866,484867.0,4.0
484867,484868.0,4.0
484868,484869.0,4.0
484869,484870.0,4.0


In [39]:
cols = ["id", "label"]
pcd_df = pd.DataFrame(pcd[:, [19, 11]], columns=cols)
pcd_df

,id,label
0,1.0,2.0
1,1.0,2.0
2,1.0,2.0
3,1.0,2.0
4,1.0,2.0
...,...,...
6739441,484869.0,4.0
6739442,484869.0,4.0
6739443,484869.0,4.0
6739444,484870.0,4.0


In [42]:
unfold_pcd = pcd_df.merge(subsampled_pcd_df, on="id", how="inner")
unfold_pcd

,id,label,y_pred
0,1.0,2.0,2.0
1,1.0,2.0,2.0
2,1.0,2.0,2.0
3,1.0,2.0,2.0
4,1.0,2.0,2.0
...,...,...,...
6739441,484869.0,4.0,4.0
6739442,484869.0,4.0,4.0
6739443,484869.0,4.0,4.0
6739444,484870.0,4.0,4.0
